In [ ]:
!unzip /content/A.zip

Streaming output truncated to the last 5000 lines.
  inflating: A/train/730889_mask_9.png  
  inflating: A/train/181447_mask_15.png  
  inflating: A/train/513968_mask_6.png  
  inflating: A/train/400179_mask_8.png  
  inflating: A/train/888263_mask_15.png  
  inflating: A/train/402209_mask_7.png  
  inflating: A/train/321724_sat_1.jpg  
  inflating: A/train/127976_sat_1.jpg  
  inflating: A/train/34330_sat_4.jpg  
  inflating: A/train/684377_mask_2.png  
  inflating: A/train/471930_sat_13.jpg  
  inflating: A/train/326238_sat_10.jpg  
  inflating: A/train/518833_sat_5.jpg  
  inflating: A/train/904606_mask_12.png  
  inflating: A/train/942594_sat_13.jpg  
  inflating: A/train/397137_sat_14.jpg  
  inflating: A/train/160037_sat_9.jpg  
  inflating: A/train/577164_sat_5.jpg  
  inflating: A/train/950926_sat_1.jpg  
  inflating: A/train/978039_mask_1.png  
  inflating: A/train/81039_sat_10.jpg  
  inflating: A/train/875327_sat_13.jpg  
  inflating: A/train/458776_sat_7.jpg  
  inflating: 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# initialization
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from google.colab import drive
import os
drive.mount('/content/drive')
import numpy as np
import time
import pandas as pd
from matplotlib.image import imread
from torchvision.io import read_image

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# path declare
master_path = '/content/drive/MyDrive/APS360Project/archive/cropped_dataset/'
csv_path = master_path + "cropped_proportion.csv"
train_csv_path = master_path + "cropped_proportion_train.csv"
val_csv_path = master_path + "cropped_proportion_valid.csv"
test_csv_path = master_path + "cropped_proportion_test.csv"
class_dict_path = master_path + "class_dict.csv"
image_folder_path = '/content/A'
transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
         transforms.Resize((256,256))])

total_info = pd.read_csv(csv_path)
train_info = pd.read_csv(train_csv_path)
val_info = pd.read_csv(val_csv_path)
test_info = pd.read_csv(test_csv_path)

In [ ]:
print(len(total_info['image_id']))
print(len(train_info['image_id']), len(train_info['image_id'])/len(total_info['image_id']))
print(len(val_info['image_id']), len(val_info['image_id'])/len(total_info['image_id']))
print(len(test_info['image_id']), len(test_info['image_id'])/len(total_info['image_id']))

12848
9616 0.7484433374844334
1616 0.12577833125778332
1616 0.12577833125778332


In [ ]:
classes = pd.read_csv(class_dict_path)
list(classes['name'])

['urban_land',
 'agriculture_land',
 'rangeland',
 'forest_land',
 'water',
 'barren_land',
 'unknown']

In [ ]:
# customed dataset for load in data
class data_full(object):
    def __init__(self, csv_info, class_dict_path, image_folder_path, transform=None):
      self.img_info = csv_info
      self.image_folder_path = image_folder_path
      self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def getitem(self, i):
        img_path = os.path.join(self.image_folder_path, self.img_info['sat_image_path'][i])
        image = read_image(img_path)
        label = [self.img_info['urban_land'][i], self.img_info['agriculture_land'][i], self.img_info['rangeland'][i],
                 self.img_info['forest_land'][i], self.img_info['water'][i], self.img_info['barren_land'][i],
                 self.img_info['unknown'][i]]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
# load to each set
train_set = []
val_set = []
test_set = []

train_dataset = data_full(train_info, class_dict_path, image_folder_path ,transform=transform)
val_dataset = data_full(val_info, class_dict_path, image_folder_path ,transform=transform)
test_dataset = data_full(test_info, class_dict_path, image_folder_path ,transform=transform)

for i in range(len(train_info['image_id']/3)):
  image, label = train_dataset.getitem(i)
  train_set.append((image, label))
  print(i)
print("finish val set")

finish train set


In [ ]:
for i in range(len(val_info['image_id'])):
  val_set.append([val_dataset.getitem(i)])
print("finish val set")

for i in range(len(test_info['image_id'])):
  test_set.append([test_dataset.getitem(i)])
print("finish test set")

finish val set
finish test set


In [ ]:
batch_size = 1
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, num_workers=1 ,
                                            shuffle=False)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, num_workers=1,
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=1,
                                            shuffle=True)

In [ ]:
def model_name(name, batch_size, learning_rate, epoch):
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

def calculate_accuracy(net, loader, criterion):
    correct = 0
    total = 0
    total_loss = 0.0
    net_eval = net.eval()
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        outputs = net_eval(inputs).T
        pred = outputs
        labels = torch.stack(labels)
        labels = labels.double()
        outputs = outputs.double()
        correct += pred.eq((labels.view(-1)).view_as(pred)).sum().item()
        total += inputs.shape[0]
        loss = criterion(outputs, labels)
        total_loss += loss.item()

    correct = correct/7
    print(correct, total)
    accuracy = correct / total
    loss = float(total_loss) / (len(loader))
    return accuracy, loss

def plot_training_curve(path):
    train_err = np.loadtxt("{}_train_err.csv".format(path))
    val_err = np.loadtxt("{}_val_err.csv".format(path))
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    plt.title("Train vs Validation Error")
    n = len(train_err) # number of epochs
    plt.plot(range(1,n+1), train_err, label="Train")
    plt.plot(range(1,n+1), val_err, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.legend(loc='best')
    plt.show()
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,n+1), train_loss, label="Train")
    plt.plot(range(1,n+1), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()

In [ ]:
def train_net(net, learning_rate=None, num_epochs=None, train_loader=None, val_loader=None, test_loader=None):
    target_classes = ['urban_land',
                      'agriculture_land',
                      'rangeland',
                      'forest_land',
                      'water',
                      'barren_land',
                      'unknown']
    torch.manual_seed(1000)  # Duplicate result or not -> set seeds random number generators
    criterion = torch.nn.BCELoss()  # Binary Cross Entropy Loss in PyTorch
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)

    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)

    print("Training Start:")
    start_time = time.time()
    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_train_err = 0.0
        total_epoch = 0
        for i, data in enumerate(train_loader, 0):
            # Get the inputs and labels
            inputs = data[0]  # Features tensor
            label_probs = data[1]  # Label probabilities list
            labels = torch.stack(label_probs)
            labels = torch.tensor(labels, requires_grad=False)  # Convert label probabilities to PyTorch tensor
            optimizer.zero_grad()
            # Forward pass, backward pass, and optimize
            outputs = net(inputs).T
            labels = labels.double()
            outputs = outputs.double()
            outputs = torch.sigmoid(outputs)
            loss = criterion(outputs, labels)  # Use PyTorch loss function
            loss.backward()
            optimizer.step()

        train_err[epoch], train_loss[epoch] = calculate_accuracy(net, train_loader, criterion)
        val_err[epoch], val_loss[epoch] = calculate_accuracy(net, val_loader, criterion)
        print(("Epoch {}: Train err: {}, Train loss: {} |" +
               "Validation err: {}, Validation loss: {}").format(
            epoch + 1,
            train_err[epoch],
            train_loss[epoch],
            val_err[epoch],
            val_loss[epoch]))
        # Save the current model (checkpoint) to a file
        model_path = model_name(net.name, batch_size, learning_rate, epoch)
        # torch.save(net.state_dict(), model_path)
    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
    # Write the train/test loss/err into CSV file for plotting later
    epochs = np.arange(1, num_epochs + 1)
    np.savetxt("{}_train_err.csv".format(model_path), train_err)
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_err.csv".format(model_path), val_err)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)

In [ ]:
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.name = "ANN"
        self.layer1 = nn.Linear(28*28, 60)
        self.layer2 = nn.Linear(60, 1)

    def forward(self, x):
        flattened = x.view(x.size(0), -1)
        activation1 = self.layer1(flattened)
        activation1 = F.relu(activation1)
        activation2 = self.layer2(activation1)
        x = activation2.squeeze(1) # Flatten to [batch_size]
        return x

In [ ]:
model = ANN()
train_net(model, learning_rate=0.002, num_epochs=3, train_loader = train_loader, val_loader = val_loader, test_loader = test_loader)
model_path = model_name("1.0", batch_size=64, learning_rate=0.002, epoch=3)
plot_training_curve(model_path)

Training Start:


<ipython-input-24-1d5fa29da7b7>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, requires_grad=False)  # Convert label probabilities to PyTorch tensor


RuntimeError: mat1 and mat2 must have the same dtype, but got Byte and Float